In [ ]:
%matplotlib inline
import copy
import numpy as np
import os
import pandas as pd
import holoviews as hv
from daq.pico import CSV
from easier import ParamState, shade, Item
from harmonic_analyzer import HarmonicAnalyzer
from harmonic import Harmonic
from pandashells.lib.lomb_scargle_lib import lomb_scargle
from scipy.fftpack import fft, fftshift, ifftshift
hv.extension('bokeh')
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from scipy.optimize import fmin_powell
from scipy import signal
import seaborn as sns
# import matplotlib as mpl
from pylab import rcParams
sns.set_context('talk')
sns.set(style='darkgrid')


import pylab as pl
pl.rcParams['figure.figsize'] = 11, 8.5


In [ ]:
%opts Curve [width=700, height=400 show_grid=True tools=['hover']]
%opts Scatter (size=5,)
%opts RGB [width=700, height=400]


In [ ]:
test_log_file = '20180118_test_log.csv'
data_dir = './20180118//'
drive_freq = 25
channel_mapper= dict(
    a='sig_gen',
    b='res_volt',
    c='rec_volt',
    d='sec_volt'
)
df_log = pd.read_csv(test_log_file).drop(['test_no', 'excitor_position'], axis=1)
df_log = df_log.rename(columns=dict(exciter_rel_to_crack='pos'))
df_log = df_log.set_index(['sample', 'pos']).sort_index()[['filename']]

df_log

In [ ]:
"""
Care about
amp/phase of primary current to secondary voltage
frac of power in the 3rd harmonic of secondary
"""

rec_list = []
for (sample, pos), file_name_ser in df_log.iterrows():
    file_name = os.path.join(data_dir, file_name_ser[0])
    print(file_name)
    df = CSV(file_name=file_name, max_sample_freq=1e9,  **channel_mapper).df
    harmonics = [1, 3]

    # fit the primary current
    h_i_prim = Harmonic(harmonics=harmonics)
    h_i_prim.fit(df.t, df.res_volt)
    
    # fit the secondary voltage
    h_v_sec = Harmonic(harmonics=harmonics)
#     h_v_sec.fit(df.t, df.sec_volt)
    h_v_sec.fit(df.t, df.rec_volt)
    
    # compute an impedence object
    h_z = h_v_sec / h_i_prim
    
    
    # populate some results
    rec = dict(
        sample=sample,
        pos=pos,
        z_amp=h_z.amplitudes[0],
        z_phase=h_z.phases[0] * 180 / np.pi,
        micropower3_frac = 1e3 * h_v_sec.amplitudes[1] ** 2 / np.sum(h_v_sec.amplitudes ** 2)
    )
    rec_list.append(rec)
dfr = pd.DataFrame(rec_list, columns=list(rec.keys()))
dfr

In [ ]:
def plotter(*args, **kwargs):
    pl.scatter(*args, **kwargs)
    pl.plot(*args, **kwargs)

In [ ]:
g = sns.FacetGrid(dfr, row='sample', sharey=False, size=3, aspect=3)

g.map(plotter , 'pos', 'z_amp')
pl.savefig('z_amp.pdf')

In [ ]:
g = sns.FacetGrid(dfr, row='sample', sharey=False, size=3, aspect=3)

g.map(plotter , 'pos', 'z_phase')
pl.savefig('z_phase.pdf')

In [ ]:
g = sns.FacetGrid(dfr, row='sample', sharey=False, size=3, aspect=3)

    
g.map(plotter , 'pos', 'micropower3_frac')
pl.savefig('harmonic_power.pdf')

In [ ]:
dfr.head()

In [ ]:
dfx = dfr.set_index(['sample', 'pos']).unstack('sample').sort_index(axis=1)
dfx

In [ ]:
field = 'micropower3_frac'
field = 'z_amp'
dfy = dfx.loc[:, pd.IndexSlice[field]].reset_index()
dfy.columns.name = None
dfy.loc[:, 'delta'] = list(dfy.broken - dfy.virgin)
dfy.head()

In [ ]:
pl.plot(dfy.pos, dfy.delta)

In [ ]:
# # Code for plotting fits
# x = df.t
# y = df.sec_volt
# h = Harmonic(harmonics=[1, 3])
# h.fit(x, y, alpha=.0001, method='regression')
# y_fit = h.predict(x)
# resid = y - y_fit
# b, a = signal.butter(2, 0.001)
# resid_smooth = signal.filtfilt(b, a, resid, padlen=150)
# (
#     shade(hv.Curve((x, y)))
#     * shade(hv.Curve((x, y_fit), kdims=['time'], vdims=['amp']), color='red')
#     + (
#         shade(hv.Curve((x, resid), vdims=['resid']), color='blue')
#         *shade(hv.Curve((x, resid_smooth)), color='red')
#     )
        
    
# ).cols(1)